# 1. Importar librerias


Cargamos las librerias necesarias: Keras y Tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as tf

from google.colab import drive
from sklearn.model_selection import train_test_split
from keras import backend
from keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras import applications
from tensorflow.keras import optimizers

from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Conv2D, MaxPooling2D, Dense, Dropout

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 2. Preparado de los datos

Desde nuestra raiz de Google Drive, vamos a cargar el fichero .zip que ha sido proporcionado. Una vez extraido hacemos el manejo de los ficheros para poder conformar los datos de entrenamiento (70%), validacion (15%) y testing (15%). El fichero .zip consta de un .csv con el porcentaje de pixeles de cada una de las imagenes que pertenecen a comida y una carpeta con todas las imagenes. Todas las imagenes tienen unas resolucion de 640 x 480.

In [ ]:
# Montamos el Google Drive en el directorio del proyecto y descomprimios el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/DL_UNIMIB2016.zip' >> /dev/null

# Especificamos los paths al directorio que contiene las imagenes y al fichero con los porcentajes
data_path = 'UNIMIB2016/'
imgs_path = data_path + "images/"
percentages_path = data_path + "percentage.csv"

# Leemos el fichero CSV con los porcentajes
percentages = pd.read_csv(percentages_path, dtype = {"class": "float"})

Mounted at /content/gdrive


In [ ]:
# Creamos las tres particiones de datos: entrenamiento, validación y test
seed = 0
train_data, test_data = train_test_split(percentages, test_size=0.3, random_state=seed)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=seed)

# Actualizamos los indices de cada particion
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# Especificamos una serie de parámetros dependientes del conjunto de datos
x_col = 'image_name'          # nombres de las columnas en el fichero CSV
y_col = 'food_pixels'

# Generamos los batches con los datos para las tres particiones
batch_size = 64

# Hacemos una normalizacion de los datos para que los valores de las capas rgb esten en un intervalo de 0 a 1
datagen = ImageDataGenerator(rescale=1./255)
# En cuanto a los datos de entrenamiento, tambien aplicamos transformaciones para poder hacer Data Augmentation
datagenTrain = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True)

train_generator = datagenTrain.flow_from_dataframe(dataframe=train_data, directory=imgs_path, x_col=x_col, y_col=y_col,
                                              class_mode="raw", target_size=(224,224), shuffle=False,
                                              batch_size=batch_size)
val_generator = datagen.flow_from_dataframe(dataframe=val_data, directory=imgs_path, x_col=x_col, y_col=y_col,
                                          class_mode="raw", target_size=(224,224), shuffle=False,
                                          batch_size=batch_size)
test_generator = datagen.flow_from_dataframe(dataframe=test_data, directory=imgs_path, x_col=x_col, y_col=y_col,
                                             class_mode="raw", target_size=(224,224), shuffle=False,
                                             batch_size=batch_size)

Found 718 validated image filenames.
Found 154 validated image filenames.
Found 155 validated image filenames.


# 3. Construccion de modelos

Construimos las funciones necesarias para solucionar el problema de prediccion de 3 maneras distintas. Haremos Fine Tunning con 2 de los modelos, mientras que con el otro, solo entrenaremos las ultimas capas.

In [ ]:
# Definicion de la funcion de perdida rmse
def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true)))

def VGG16_FineTuningModel():
  # Cargamos el modelo preentrenado con los pesos de imagenet para hacer fine tuning
  base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

  # Añadimos nuevas capas al final para adaptar el modelo a nuestro problema
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)

  # Añadimos una última capa completamente conectada, la salida sera lineal
  predictions = Dense(1)(x)

  # Creamos el modelo final y lo compilamos
  model = Model(inputs=[base_model.input], outputs=[predictions])
  model.summary()

  model.compile(optimizer = optimizers.Adam(learning_rate=0.0001), loss = root_mean_squared_error)

  # Configuramos el earlystopping, despues de 10 epocas si no mejora, paramos el entrenamiento
  es = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode=min
  )

  # Almacenamos la epoch con mejores pesos para hacer un val_loss minimo
  mc = ModelCheckpoint(
    'best_m.h5',
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True
  )

  # Entrenamos el modelo
  model.fit(train_generator,
          epochs=70,
          verbose=2,
          callbacks=[es, mc],
          steps_per_epoch=len(train_data)/batch_size,
          validation_data=val_generator,
          validation_steps=len(val_data)/batch_size)
  return model


def VGG16_LastLayersModel():
  base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

  # Ponemos todas las capas como no entrenables, solo entrenaremos las capas finales
  for layer in base_model.layers:
    layer.trainable = False

  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)
  predictions = Dense(1)(x)

  model = Model(inputs=[base_model.input], outputs=[predictions])
  model.summary()

  model.compile(optimizer = optimizers.Adam(learning_rate=0.01), loss = root_mean_squared_error)

  es = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode=min
  )

  mc = ModelCheckpoint(
    'best_m.h5',
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True
  )

  model.fit(train_generator,
          epochs=70,
          verbose=2,
          callbacks=[es, mc],
          steps_per_epoch=len(train_data)/batch_size,
          validation_data=val_generator,
          validation_steps=len(val_data)/batch_size)
  return model


def InceptionResNetV2_FineTuningModel():
  # Cargamos el modelo preentrenado con los pesos de imagenet para hacer fine tuning
  base_model = applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)
  predictions = Dense(1)(x)

  model = Model(inputs=[base_model.input], outputs=[predictions])
  model.summary()

  model.compile(optimizer = optimizers.Adam(learning_rate=0.0001), loss = root_mean_squared_error)

  es = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode=min
  )

  mc = ModelCheckpoint(
    'best_m.h5',
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True
  )

  model.fit(train_generator,
          epochs=70,
          verbose=2,
          callbacks=[es, mc],
          steps_per_epoch=len(train_data)/batch_size,
          validation_data=val_generator,
          validation_steps=len(val_data)/batch_size)
  return model

Exception ignored in: <function _xla_gc_callback at 0x7b6f528ceb00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


# 4. Ejecucion de entrenamiento


In [ ]:
# Ejecutamos la funcion del modelo
VGG16FineTuningModel = VGG16_FineTuningModel()

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

Epoch 1/70

Epoch 1: val_loss improved from inf to 5.34755, saving model to best_m.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 - 74s - loss: 7.5327 - val_loss: 5.3476 - 74s/epoch - 7s/step
Epoch 2/70

Epoch 2: val_loss improved from 5.34755 to 3.97675, saving model to best_m.h5
11/11 - 12s - loss: 4.2465 - val_loss: 3.9767 - 12s/epoch - 1s/step
Epoch 3/70

Epoch 3: val_loss improved from 3.97675 to 3.09228, saving model to best_m.h5
11/11 - 12s - loss: 3.2100 - val_loss: 3.0923 - 12s/epoch - 1s/step
Epoch 4/70

Epoch 4: val_loss did not improve from 3.09228
11/11 - 11s - loss: 2.1959 - val_loss: 3.6142 - 11s/epoch - 968ms/step
Epoch 5/70

Epoch 5: val_loss did not improve from 3.09228
11/11 - 11s - loss: 2.6712 - val_loss: 3.6658 - 11s/epoch - 989ms/step
Epoch 6/70

Epoch 6: val_loss improved from 3.09228 to 2.03407, saving model to best_m.h5
11/11 - 12s - loss: 2.2463 - val_loss: 2.0341 - 12s/epoch - 1s/step
Epoch 7/70

Epoch 7: val_loss improved from 2.03407 to 1.89654, saving model to best_m.h5
11/11 - 12s - loss: 1.9271 - val_loss: 1.8965 - 12s/epoch - 1s/step
Epoch 8/70

Epoch 8: val_loss did not im

In [ ]:
VGG16LastLayersModel = VGG16_LastLayersModel()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

Epoch 1/70

Epoch 1: val_loss improved from inf to 4.14039, saving model to best_m.h5
11/11 - 9s - loss: 5.6273 - val_loss: 4.1404 - 9s/epoch - 832ms/step
Epoch 2/70

Epoch 2: val_loss improved from 4.14039 to 3.59665, saving model to best_m.h5
11/11 - 6s - loss: 3.5999 - val_loss: 3.5966 - 6s/epoch - 557ms/step
Epoch 3/70

Epoch 3: val_loss improved from 3.59665 to 3.20762, saving model to best_m.h5
11/11 - 7s - loss: 3.3580 - val_loss: 3.2076 - 7s/epoch - 624ms/step
Epoch 4/70

Epoch 4: val_loss improved from 3.20762 to 2.72371, saving model to best_m.h5
11/11 - 6s - loss: 2.9221 - val_loss: 2.7237 - 6s/epoch - 559ms/step
Epoch 5/70

Epoch 5: val_loss improved from 2.72371 to 2.53439, saving model to best_m.h5
11/11 - 7s - loss: 2.7340 - val_loss: 2.5344 - 7s/epoch - 615ms/step
Epoch 6/70

Epoch 6: val_loss improved from 2.53439 to 2.42050, saving model to best_m.h5
11/11 - 6s - loss: 2.5859 - val_loss: 2.4205 - 6s/epoch - 558ms/step
Epoch 7/70

Epoch 7: val_loss improved from 2.4205

In [ ]:
InceptionResNetV2FineTuningModel = InceptionResNetV2_FineTuningModel()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_406 (Conv2D)         (None, 111, 111, 32)         864       ['input_5[0][0]']             
                                                                                                  
 batch_normalization_406 (B  (None, 111, 111, 32)         96        ['conv2d_406[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_406 (Activation  (None, 111, 111, 32)         0         ['batch_normalization_40

Epoch 1/70

Epoch 1: val_loss improved from inf to 21.81758, saving model to best_m.h5
11/11 - 108s - loss: 10.2557 - val_loss: 21.8176 - 108s/epoch - 10s/step
Epoch 2/70

Epoch 2: val_loss improved from 21.81758 to 11.90337, saving model to best_m.h5
11/11 - 22s - loss: 3.1673 - val_loss: 11.9034 - 22s/epoch - 2s/step
Epoch 3/70

Epoch 3: val_loss improved from 11.90337 to 6.18550, saving model to best_m.h5
11/11 - 27s - loss: 2.0727 - val_loss: 6.1855 - 27s/epoch - 2s/step
Epoch 4/70

Epoch 4: val_loss improved from 6.18550 to 4.90515, saving model to best_m.h5
11/11 - 25s - loss: 1.8399 - val_loss: 4.9052 - 25s/epoch - 2s/step
Epoch 5/70

Epoch 5: val_loss improved from 4.90515 to 2.65763, saving model to best_m.h5
11/11 - 19s - loss: 1.7152 - val_loss: 2.6576 - 19s/epoch - 2s/step
Epoch 6/70

Epoch 6: val_loss did not improve from 2.65763
11/11 - 14s - loss: 1.7121 - val_loss: 2.7147 - 14s/epoch - 1s/step
Epoch 7/70

Epoch 7: val_loss did not improve from 2.65763
11/11 - 14s - loss

# 5. Evaluacion del modelo con datos de testing

Evaluamos a continuacion el rendimiento de los 3 modelos con los datos de testing.

In [ ]:
VGG16FineTuningModel.evaluate(test_generator, batch_size=128)
VGG16LastLayersModel.evaluate(test_generator, batch_size=128)
InceptionResNetV2FineTuningModel.evaluate(test_generator, batch_size=128)

3/3 [==============================] - 4s 2s/step - loss: 2.3650


2.3649988174438477

Para verlo de una manera mas intuitiva, imprimimos los resultados reales junto con sus valores predichos por los distintos modelos implementados.

In [ ]:
predictionsVGG16_FineTuningModel = VGG16FineTuningModel.predict(test_generator, verbose=1)
predictionsVGG16_LastLayersModel= VGG16LastLayersModel.predict(test_generator, verbose=1)
predictionsInceptionResNetV2_FineTuningModel = InceptionResNetV2FineTuningModel.predict(test_generator, verbose=1)

for i in range(0, len(test_data)):
  print("\n Ejemplo", i)
  print("\t Imagen: ", test_data.iloc[i, 0])
  print("\t Porcentaje predicho por VGG16 con Fine Tuning: ", predictionsVGG16_FineTuningModel[i])
  print("\t Porcentaje predicho por VGG16 con ultimas capas entrenadas: ", predictionsVGG16_LastLayersModel[i])
  print("\t Porcentaje predicho por InceptionResNetV2 con Fine Tuning: ", predictionsInceptionResNetV2_FineTuningModel[i])
  print("\t Porcentaje real: ", test_data.iloc[i, 1])

3/3 [==============================] - 4s 364ms/step

 Ejemplo 0
	 Imagen:  20151211_122813.jpg
	 Porcentaje predicho por VGG16 con Fine Tuning:  [18.70737]
	 Porcentaje predicho por VGG16 con ultimas capas entrenadas:  [14.828028]
	 Porcentaje predicho por InceptionResNetV2 con Fine Tuning:  [16.5607]
	 Porcentaje real:  18.09882067

 Ejemplo 1
	 Imagen:  20151211_124745.jpg
	 Porcentaje predicho por VGG16 con Fine Tuning:  [23.852016]
	 Porcentaje predicho por VGG16 con ultimas capas entrenadas:  [24.527464]
	 Porcentaje predicho por InceptionResNetV2 con Fine Tuning:  [18.494036]
	 Porcentaje real:  24.47016822

 Ejemplo 2
	 Imagen:  20151204_130806.jpg
	 Porcentaje predicho por VGG16 con Fine Tuning:  [14.851699]
	 Porcentaje predicho por VGG16 con ultimas capas entrenadas:  [13.761499]
	 Porcentaje predicho por InceptionResNetV2 con Fine Tuning:  [11.920139]
	 Porcentaje real:  14.56150679

 Ejemplo 3
	 Imagen:  20151211_130428.jpg
	 Porcentaje predicho por VGG16 con Fine Tuning: 